# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import json
import re

pd.set_option('display.max_colwidth', None)

Mounted at /content/drive


# Load responses

In [ ]:
def count_ids_in_txt(content:str)->None:

  suffixes = ["ins", "inp", "out"]

  counts = {}

  for suffix in suffixes:

      pattern = fr'["\']ID["\']:\s*["\']\d+{suffix}["\']'
      matches = re.findall(pattern, content)
      counts[suffix] = len(matches)
      print(f"Total de IDs com sufixo '{suffix}': {counts[suffix]}")

  # Soma o total de IDs
  total_ids = sum(counts.values())
  print("Quantidade total de IDs:", total_ids)

## Format in buckets

In [ ]:
input_paths_b = [
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/bucket/g1_response.txt",
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/bucket/g2_response.txt",
]

output_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/bucket/gr_bucket.txt"

line_counts = []

with open(output_path, 'w', encoding='utf-8') as output_file:
    for file_path in input_paths_b:
        with open(file_path, 'r', encoding='utf-8') as input_file:
            lines = input_file.readlines()
            line_counts.append(len(lines))
            output_file.write("".join(lines) + "\n")


with open(output_path, 'r', encoding='utf-8') as output_file:
    count_merged = sum(1 for _ in output_file)

with open(output_path, 'r', encoding='utf-8') as file:#output_path
    content_b = file.read()

count_ids_in_txt(content_b)

Total de IDs com sufixo 'ins': 57491
Total de IDs com sufixo 'inp': 17829
Total de IDs com sufixo 'out': 49032
Quantidade total de IDs: 124352


In [ ]:
def extrair_dados_para_dataframe_by_bucket(content):
    # Dividir o conteúdo por blocos de dados
    blocos_dados = content.split('json\n[')
    blocos_dados = [bloco.replace('\u200b', '').strip() for bloco in blocos_dados]

    dados_extraidos = []

    # Função para extrair dados dos novos blocos de texto com colunas ID, PT e GLOSA
    def extrair_dados_multiplos(texto):
        try:
            # Tenta carregar o texto diretamente como JSON, caso ele já esteja em um formato válido
            dados = json.loads(texto)
            if isinstance(dados, list):  # Verifica se é uma lista de dicionários
                return [(item.get("ID"), item.get("PT"), item.get("GLOSA")) for item in dados]
        except json.JSONDecodeError:
            # Caso falhe em decodificar, utiliza regex para buscar blocos que não estão no formato JSON
            padrao = re.compile(
                r'"ID":\s*"([^"]+)"\s*,\s*"PT":\s*"(.*?)"\s*,\s*"(?:GLOSA|GLOSSARY|GLOSE)":\s*"(.*?)"',
                re.DOTALL
            )
            matches = padrao.findall(texto)
            return [(match[0], match[1], match[2] if match[2] != 'null' else None) for match in matches]

        return []

    # Extrair dados de cada bloco, ignorando o primeiro elemento vazio
    for bloco in blocos_dados[1:]:
        dados_extraidos.extend(extrair_dados_multiplos(bloco))

    # Criar o DataFrame diretamente com a lista de dados extraídos
    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

df_bucket = extrair_dados_para_dataframe_by_bucket(content_b)
print(f"Bucket: shape : {df_bucket.shape}\nIds únicos: {df_bucket.ID.nunique()}")
df_bucket.head(2)

Bucket: shape : (123217, 3)
Ids únicos: 118207


,ID,PT,GLOSA
0,0ins,Dê três dicas para se manter saudável.,VOCÊ DAR TRÊS DICA SAÚDE MANTER?
1,2ins,Descreva a estrutura de um átomo.,ÁTOMO ESTRUTURA DESCREVER.


## Format in single

In [ ]:
input_paths_s = [
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/single/g3_response.txt",
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/single/g4_response.txt",
    "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/single/glosa_try_again_response.txt",
]

output_path = "/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/GLOSA/single/gr_single.txt"

line_counts = []

with open(output_path, 'w', encoding='utf-8') as output_file:
    for file_path in input_paths_s:
        with open(file_path, 'r', encoding='utf-8') as input_file:
            lines = input_file.readlines()
            line_counts.append(len(lines))
            output_file.write("".join(lines) + "\n")

with open(output_path, 'r', encoding='utf-8') as output_file:
    count_merged = sum(1 for _ in output_file)

with open(output_path, 'r', encoding='utf-8') as file:#output_path
    content_by_one = file.read()

count_ids_in_txt(content_by_one)

Total de IDs com sufixo 'ins': 1775
Total de IDs com sufixo 'inp': 159
Total de IDs com sufixo 'out': 365
Quantidade total de IDs: 2299


In [ ]:
'''Total de IDs com sufixo 'ins': 755
Total de IDs com sufixo 'inp': 32
Total de IDs com sufixo 'out': 124
Quantidade total de IDs: 911'''

"Total de IDs com sufixo 'ins': 755\nTotal de IDs com sufixo 'inp': 32\nTotal de IDs com sufixo 'out': 124\nQuantidade total de IDs: 911"

In [1]:
def extrair_dados_para_dataframe_single_block(content):
    content = content.replace("json\n", "").strip()
    pattern = re.compile(
        r"\{\s*'ID':\s*'([^']+)',\s*'PT':\s*'([^']+)',\s*'GLOSA':\s*'([^']*)'\s*\}"
    )

    dados_extraidos = pattern.findall(content)

    df = pd.DataFrame(dados_extraidos, columns=['ID', 'PT', 'GLOSA'])
    return df

df_by_one = extrair_dados_para_dataframe_single_block(content_by_one)
print(f"Bucket: shape : {df_by_one.shape}\nIds únicos: {df_by_one.ID.nunique()}")
df_by_one.head(2)

NameError: name 'content_by_one' is not defined

## Concat and remove duplicates

In [ ]:
ids_duplicados = df_bucket[df_bucket.ID.duplicated()].ID.unique()
print("Buckets - IDs duplicados:", len(ids_duplicados))

ids_duplicados = df_by_one[df_by_one.ID.duplicated()].ID.unique()
print("Singles - IDs duplicados:", len(ids_duplicados))

Buckets - IDs duplicados: 4968
Singles - IDs duplicados: 445


In [ ]:
df_bucket['count_glosa'] = df_bucket['GLOSA'].apply(lambda x: len(x))
df_by_one['count_glosa'] = df_by_one['GLOSA'].apply(lambda x: len(x))

df_concat = pd.concat([df_bucket, df_by_one], axis=0)
df_concat.head(2)

,ID,PT,GLOSA,count_glosa
0,0ins,Dê três dicas para se manter saudável.,VOCÊ DAR TRÊS DICA SAÚDE MANTER?,32
1,2ins,Descreva a estrutura de um átomo.,ÁTOMO ESTRUTURA DESCREVER.,26


In [ ]:
df_concat = df_concat.sort_values(by='count_glosa', ascending=False )
df_concat.head(1)

,ID,PT,GLOSA,count_glosa
72881,15908out,"Artigo nº 1\nO que é criptomoeda?\n\nCriptomoeda é uma forma digital de moeda descentralizada, o que significa que não está vinculada a nenhum governo ou instituição financeira. Ela é criada e trocada usando um computador ou sistema de contabilidade digital, chamado blockchain. A criptomoeda é usada para comprar itens e serviços, ou para investir, da mesma forma que o dinheiro físico é usado no mundo real. Alguns dos tipos mais populares de criptomoeda incluem Bitcoin, Ethereum, Litecoin e Ripple.\n\nCriptomoeda é um tipo de ativo digital, projetado para funcionar como um meio de troca seguro. As criptomoedas usam controle descentralizado, o que significa que as transações são gerenciadas por redes de computadores localizados ao redor do mundo, em vez de serem regulamentadas por uma autoridade central como um sistema bancário tradicional. A criptomoeda é criada por meio de um processo especial chamado mineração, no qual os computadores resolvem equações matemáticas complexas para validar as transações dentro da rede. Como recompensa por sua computação, os mineradores são recompensados com a criptomoeda de sua escolha.\n\nArtigo nº 2\nVantagens da criptomoeda\n\nA criptomoeda oferece aos usuários uma série de vantagens sobre os métodos tradicionais de pagamento e câmbio. As criptomoedas são seguras, pois são criptografadas com uma assinatura digital verificável e não estão sujeitas à manipulação por instituições terceirizadas. As transações de criptomoeda são rápidas e convenientes, sem a necessidade de um intermediário terceirizado. Por ser descentralizada, representa uma forma de moeda global, sem banco central ou governo associado ao seu uso. Além disso, a criptomoeda não exige que o usuário compartilhe informações pessoais e oferece aos usuários mais privacidade e controle do que outros métodos de pagamento.\n\nArtigo nº 3\nDesvantagens da criptomoeda\n\nApesar das inúmeras vantagens da criptomoeda, há certas desvantagens que também devem ser levadas em consideração. As criptomoedas não são amplamente aceitas pelos varejistas, o que significa que não podem ser usadas para comprar a maioria dos bens e serviços. Além disso, as transações de criptomoeda não são reversíveis, o que significa que se um usuário acidentalmente enviou criptomoeda para o endereço errado ou cometeu um erro, ele não tem como recuperar os fundos. As transações de criptomoedas também são vulneráveis a potenciais ameaças de segurança e fraudes, devido à sua natureza digital. Finalmente, devido aos mercados de criptomoedas em constante flutuação, os investidores podem sofrer perdas substanciais em seus investimentos.\n\nArtigo nº 4\nComo investir em criptomoedas\n\nInvestir em criptomoedas é um processo bastante simples, mas há certas considerações que devem ser feitas. Primeiro, os investidores devem pesquisar e se familiarizar com os vários tipos de criptomoedas disponíveis e determinar quais opções melhor atendem aos seus objetivos e situação financeira. Uma vez feita a escolha, o investidor pode comprar sua criptomoeda de escolha usando uma bolsa de criptomoedas, como Coinbase ou Bittrex. O investidor pode então armazenar sua criptomoeda em uma carteira digital, como uma carteira de hardware ou carteira quente, dependendo da quantia armazenada e de sua tolerância pessoal ao risco. Alternativamente, um investidor pode comprar um fundo ou fundo de criptomoeda, o que lhe permite investir em uma \""cesta\"" de diferentes criptomoedas sem ter que gerenciá-las individualmente. Finalmente, os investidores devem sempre se lembrar de diversificar seu portfólio com outras classes de ativos e permanecer informados sobre os últimos desenvolvimentos nos mercados de criptomoedas para tomar decisões informadas.",ARTIGO NÚMERO 1 O QUE SER CRIPTOMOEDA CRIPTOMOEDA SER FORMA DIGITAL MOEDA DESCENTRALIZADO SIGNIFICAR NÃO ESTAR VINCULADO NENHUM GOVERNO INSTITUIÇÃO FINANCEIRO ELA SER CRIAR TROCAR USAR COMPUTADOR SISTEMA CONTABILIDADE 

In [ ]:
print(f"Forma inicial: {df_concat.shape}, IDs únicos: {df_concat.ID.nunique()}")
df_concat = df_concat.drop_duplicates(subset='ID').sort_index(ascending=True)
print(f"Forma após remoção de duplicatas: {df_concat.shape}, IDs únicos: {df_concat.ID.nunique()}")
df_concat.head()

Forma inicial: (125473, 4), IDs únicos: 118315
Forma após remoção de duplicatas: (118315, 4), IDs únicos: 118315


,ID,PT,GLOSA,count_glosa
0,0ins,Dê três dicas para se manter saudável.,VOCÊ DAR TRÊS DICA SAÚDE MANTER?,32
1,2ins,Descreva a estrutura de um átomo.,ÁTOMO ESTRUTURA DESCREVER.,26
2,3ins,Como podemos reduzir a poluição do ar?,COMO PODER REDUZIR POLUIÇÃO AR?,31
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,VOCÊ DESCREVER OCASIÃO VOCÊ TER QUE TOMAR DECISÃO DIFÍCIL.,58
4,5ins,Identifique o estranho.,IDENTIFICAR ESTRANHO.,21


In [ ]:
df_concat.query("ID == '12370ins'")

,ID,PT,GLOSA,count_glosa
10139,12370ins,Procure a biografia de Malcolm X na Wikipédia.,WIKIPÉDIA MALCOLM X BIOGRAFIA PROCURAR.,39


In [ ]:
df_concat.query("ID == '1ins'")

,ID,PT,GLOSA,count_glosa


# Resultados Compilados GLOSA

In [ ]:
df_concat[['ID', 'PT', 'GLOSA']].to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/OUTPUT/glosa_compiled_results.csv", index=False)

# Erros

Os gerados pela glos, em output
Os que não foram convertidos em glosa

## Output error glosa

In [ ]:
glosa_errors = df_concat[df_concat['count_glosa'] < 5]
print(f"Glosa again by error {glosa_errors.shape}\nIds únicos :{glosa_errors.ID.nunique()}")
glosa_errors.head()

Glosa again by error (154, 4)
Ids únicos :154


,ID,PT,GLOSA,count_glosa
299,41350ins,erro,ERRO,4
901,20165out,"<span style=\\\\""font-style: italic; font-size: larger;",,1
1758,50530ins,"Crie uma situação do mundo real com as palavras ""torta"", ""caderno"" e ""centavo""",NADA,4
1795,6899inp,Git,GIT,3
1801,11328inp,Ema,EMA,3


In [ ]:
df_concat['GLOSA'].value_counts()

,count
GLOSA,
FRASE \,150
PALAVRA \,40
\,37
NADA.,26
ROMANCE \,20
...,...
CENA DETALHE DESCREVER NARRADOR ONISCIENTE SER.,1
MUDANÇA CLIMÁTICA IMPACTO REDUZIR PLANO ABRANGENTE GERAR.,1
TEORIA CAOS IDEIA USAR CENÁRIO DESCREVER.,1


## Errors in base data

Os que de fato serviram de input para glosa (pós figurativos)

In [ ]:
df_base = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/INPUT/formated_glosa_to_input.csv")
df_base.head()

,ID,PT,TOKENS_EST_INSTANCE,TOKENS_EST_ID
0,0ins,Dê três dicas para se manter saudável.,16,2
1,2ins,Descreva a estrutura de um átomo.,15,2
2,3ins,Como podemos reduzir a poluição do ar?,16,2
3,4ins,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,28,2
4,5ins,Identifique o estranho.,8,2


In [ ]:
erros_not_convert = df_base[~df_base['ID'].isin(df_concat['ID'])]
erros_not_convert.shape, erros_not_convert.ID.nunique()

((14, 4), 14)

In [ ]:
erros_not_convert

,ID,PT,TOKENS_EST_INSTANCE,TOKENS_EST_ID
45120,51501ins,Faça uma lista de atividades para um encontro romântico.,20,3
92456,35067out,"Eu me solidarizo com o movimento que defende que vidas negras importam porque vidas negras importam. Precisamos usar nossas vozes, energia e recursos para apoiar os direitos e a dignidade de todos.",68,3
98105,41210out,"Um exemplo de obra de literatura afro-americana é Beloved, de Toni Morrison. Ele conta a história da luta de uma mulher negra para superar seu passado traumático.",55,3
105425,49277out,Esta substância contém sódio e cloreto.,18,3
105429,49282out,"Um dos meus livros favoritos de ficção para jovens adultos é The Hate U Give, de Angie Thomas. Ele conta a história de uma jovem negra nos Estados Unidos lidando com raça, brutalidade policial, identidade e família. É um livro inteligente e que causa impacto, perfeito para adolescentes e adultos.",96,3
105434,49287out,"John e Jane podem gostar de uma visita a um restaurante italiano local. Algumas opções populares na área incluem Buon Appetito e La Trattoria. Ambos os restaurantes têm deliciosos pratos de massa e extensas listas de vinhos. Eles também oferecem mesas ao ar livre, tornando-os uma escolha perfeita para uma noite romântica.",119,3
118313,22650out,A gravidade é tão forte que distorce a estrutura do espaço-tempo e até mesmo a luz fica presa dentro de seus limites.,45,3
118314,39228out,"Sua atração gravitacional é tão forte que a velocidade da luz não é suficiente para superar a força gravitacional, tornando-o completamente preto e invisível.",57,3
118315,39783inp,"\""O serviço tem resultados de A e B\""",14,4
118316,47309inp,"\""Foi agradável trabalhar com você.\""",15,4


## Join ERROS

In [ ]:
glosa_errors.shape[0] + erros_not_convert.shape[0]

1413

In [ ]:
erros_final = pd.concat([glosa_errors[['ID', 'PT']], erros_not_convert[['ID', 'PT']]], axis=0)
print(f"Compiled errors: {erros_final.shape}\n ids únicos: {erros_final.ID.nunique()}")
erros_final.head()

Compiled errors: (1413, 2)
 ids únicos: 1413


,ID,PT
17,19ins,"Gere três verbos que significam o mesmo que \""pedir desculpas\"""
106,108ins,"Pense em uma frase ou expressão idiomática que contenha a palavra \""dormir\""."
118,143ins,"Escreva uma definição simples da palavra \""economia\""."
182,207ins,"Qual é a contração de \\\""they are\\\""?"
265,311ins,"Escreva uma definição de \\\""photoshop\\\""."


In [ ]:
#erros_final[['ID', 'PT']].to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/INPUT/glosa_try_again.csv", index=False)

# Format results to original format

['instruction', 'input', 'output']

In [ ]:
df_concat[['ID', 'TYPE']] = df_concat['ID'].str.extract(r'^(.*?)(ins|inp|out)$', expand=True)
df_concat = df_concat.astype({'ID':int})
df_concat = df_concat.sort_index(ascending=True)
print(f"Resultado glosa---\n shape: {df_concat.shape}\n ids únicos: {df_concat.ID.nunique()}")
df_concat.head()

Resultado glosa---
 shape: (118307, 5)
 ids únicos: 51982


,ID,PT,GLOSA,count_glosa,TYPE
0,0,Dê três dicas para se manter saudável.,VOCÊ DAR TRÊS DICA SAÚDE MANTER?,32,ins
1,2,Descreva a estrutura de um átomo.,ÁTOMO ESTRUTURA DESCREVER.,26,ins
2,3,Como podemos reduzir a poluição do ar?,COMO PODER REDUZIR POLUIÇÃO AR?,31,ins
3,4,Descreva uma ocasião em que você teve que tomar uma decisão difícil.,VOCÊ DESCREVER OCASIÃO VOCÊ TER QUE TOMAR DECISÃO DIFÍCIL.,58,ins
4,5,Identifique o estranho.,IDENTIFICAR ESTRANHO.,21,ins


In [ ]:
# Separando cada um de acordo com o seu tipo

df_inp = df_concat[df_concat['TYPE'] == 'inp'][['ID', 'GLOSA']].rename(columns={'GLOSA':'input'})
df_ins = df_concat[df_concat['TYPE'] == 'ins'][['ID', 'GLOSA']].rename(columns={'GLOSA':'instruction'})
df_out = df_concat[df_concat['TYPE'] == 'out'][['ID', 'GLOSA']].rename(columns={'GLOSA':'output'})

df_merged = pd.merge(df_ins, df_inp, on='ID', how='outer')
df_merged = pd.merge(df_merged, df_out, on='ID', how='outer')
df_merged = df_merged.sort_values(by='ID', ascending=True)

print(f"Resultado glosa---\n shape: {df_merged.shape}\n ids únicos: {df_merged.ID.nunique()}")
df_merged.head()

Resultado glosa---
 shape: (51982, 4)
 ids únicos: 51982


,ID,instruction,input,output
0,0,VOCÊ DAR TRÊS DICA SAÚDE MANTER?,NaN,1. COMER DIETA BALANCEADO INCLUIR MUITO FRUTA VEGETAL 2. EXERCITAR REGULARMENTE MANTER CORPO ATIVO FORTE 3. DORMIR SUFICIENTE MANTER HORÁRIO SONO CONSISTENTE
1,2,ÁTOMO ESTRUTURA DESCREVER.,NaN,NÚCLEO ÁTOMO TER PRÓTON NÊUTRON ELÉTRON VIAJAR ÓRBITA AO REDOR NÚCLEO. PRÓTON NÊUTRON CARGA POSITIVO ELÉTRON CARGA NEGATIVO RESULTADO ÁTOMO NEUTRO. PARTÍCULA CADA NÚMERO DETERMINAR NÚMERO ATÔMICO TIPO ÁTOMO.
2,3,COMO PODER REDUZIR POLUIÇÃO AR?,NaN,MODO REDUZIR POLUIÇÃO AR MUITO EXISTIR COMO MUDAR ENERGIA RENOVÁVEL USAR TRANSPORTE PÚBLICO INCENTIVAR QUEIMA COMBUSTÍVEL FÓSSIL PROIBIR POLÍTICA REDUZIR EMISSÃO FONTE INDUSTRIAL IMPLEMENTAR PADRÃO EMISSÃO VEÍCULO IMPLEMENTAR. INDIVÍDUO PODE FAZER PARTE REDUZIR POLUIÇÃO AR USAR CARRO REDUZIR QUEIMAR MADEIRA EVITAR APARELHO EFICIÊNCIA ENERGÉTICA MUDAR.
3,4,VOCÊ DESCREVER OCASIÃO VOCÊ TER QUE TOMAR DECISÃO DIFÍCIL.,NaN,EU TRABALHAR GERENTE PROJETO CONSTRUTOR DECISÃO DIFÍCIL TOMAR. EU RESPONSÁVEL PROJETO CONCLUIR DATA CLIENTE EXPECTATIVA ATENDER. ATRASO INESPERADO CONCLUIR PRAZO NÃO PODE. DECISÃO DIFÍCIL TOMAR. EU PRAZO ESTENDER RECURSO EQUIPE ESTICAR ORÇAMENTO AUMENTAR. DECISÃO ARRISCADO MAS EU CONTINUAR DECIDIR PROJETO PRAZO CONCLUIR CLIENTE EXPECTATIVA ATENDER. PROJETO CONCLUIR SUCESSO EU VER CAPACIDADE LIDERANÇA DECISÃO.
4,5,IDENTIFICAR ESTRANHO.,TWITTER INSTAGRAM TELEGRAM.,TELEGRAMA.


In [ ]:
# depois desse, juntar com os que não foram no figurative
#df_merged.to_csv("/content/drive/MyDrive/Colab Notebooks/DATASETS/GEMINI/chat_format_glosa_complete.csv", index=False)